In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from glob import glob

files = glob('url/*.json')
len(files)

2022

In [23]:
urls = []
for f in files:
    with open(f) as fopen:
        urls.extend(json.load(fopen))
        
urls = sorted(set(urls))
len(urls)

46479

In [6]:
r = requests.get(urls[0])

In [10]:
urls[0]

'https://malaysiaindru.my/100007'

In [7]:
soup = BeautifulSoup(r.content, "lxml")

In [12]:
soup.find('h2').text

'உ.பி. கலவரத்தின் பின்னணியில் அரசியல் சதி!'

In [14]:
soup.find('div', {'class': 'entry-content'}).get_text(separator="\n").strip()

'உத்தரப் பிரதேசத்தில் நடைபெற்ற கலவரத்தின் பின்னணியில் அரசியல் சதி இருக்கலாம் என்று மத்திய உள்துறை அமைச்சர் சுஷீல்குமார் ஷிண்டே சூசகமாகத் தெரிவித்தார்.\nமாநிலத்தின் முசாஃபர்நகர் மற்றும் அதன் அண்டை மாவட்டங்களில் இரு வகுப்பினருக்கு இடையே கடந்த சனிக்கிழமை கலவரம் மூண்டது. இதில் உயிரிழந்தவர்களின் எண்ணிக்கை 40ஆகியுள்ளது. வன்முறையில் காயமடைந்தவர்கள் பல்வேறு மருத்துவமனைகளில் சிகிச்சை பெற்று வருகின்றனர்.\nஇந்நிலையில், உத்தரப் பிரதேசத்தில் நடைபெற்ற கலவரத்தின் பின்னணியில் அரசியல் சதி உள்ளதாக மாநில முதல்வர் அகிலேஷ் யாதவ் கூறியது குறித்து மத்திய உள்துறை அமைச்சர் சுஷீல்குமார் ஷிண்டேவிடம் தில்லியில் செய்தியாளர்கள் புதன்கிழமை கேட்டனர். அதற்குப் பதிலளித்து அவர் கூறியது:\nமுசாஃபர்நகரில் நடைபெற்ற வன்முறை குறித்து முழுமையான அறிக்கை வரும் வரை அரசியல் சதி குறித்து என்னால் கருத்து கூற இயலாது. ஆனால், அத்தகைய சதியில் அரசியல் கட்சிகள் ஈடுபட்டிருக்கலாம்.\nமக்களவை பொதுத் தேர்தல் நெருங்கி வருவதை முன்னிட்டு, மதக் கலவரம் ஏற்படலாம் என்று 11 மாநிலங்களை மத்திய அரசு எச்சரித்திருந்தது. கலவலத்தால் பாதிக்கப்பட்ட மேற்கு உத்த

In [15]:
!mkdir page

In [29]:
from concurrent.futures import ThreadPoolExecutor, as_completed
max_worker = 10

def get_url(url, i):
    
    filename = os.path.join('page', f'{i}.json')
    if os.path.exists(filename):
        return
    
    try:
        
        r = requests.get(urls[i])
        soup = BeautifulSoup(r.content, "lxml")

        data = {
            'url': urls[i],
            'title': soup.find('h2').text.strip(),
            'body': soup.find('div', {'class': 'entry-content'}).get_text(separator="\n").strip()
        }

        with open(filename, 'w') as fopen:
            fopen.write(f'{json.dumps(data)}\n')
    except:
        pass

In [30]:
for i in tqdm(range(0, len(urls), max_worker)):
    batch = urls[i: i + max_worker]
    batch = [(urls[i + k], i + k) for k in range(len(batch))]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, *url): url for url in batch}

        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 4648/4648 [5:31:41<00:00,  4.28s/it]


In [31]:
from glob import glob

files = glob('page/*')
len(files)

46453

In [32]:
with open(files[0]) as fopen:
    data = json.load(fopen)

In [33]:
data

{'url': 'https://malaysiaindru.my/189633',
 'title': 'இன்று 2,253 புதிய நோய்த்தொற்றுகள், பேராக்கில் பாதிப்புகளின் எண்ணிக்கை அதிகரிப்பு',
 'body': 'கோவிட் 19\n | நாட்டில் இன்று நண்பகல் வரையில், மொத்தம் 2,253 கோவிட் -19 புதிய நோய்த்தொற்றுகள் பதிவாகியுள்ளதாக சுகாதார அமைச்சு தெரிவித்துள்ளது.\nபெரும்பாலான பாதிப்புகள் கிள்ளான் பள்ளத்தாக்கில் (755 நேர்வுகள்; 33.5 விழுக்காடு) பதிவாகியுள்ளன. அதேவேளையில், பேராக்கிலும் பாதிப்புகளின் எண்ணிக்கையில் (545 நேர்வுகள்; 24.2 விழுக்காடு) அதிகரிப்பு ஏற்பட்டுள்ளது.\nஇதற்கிடையில், 11 புதிய இறப்புகள் இன்று பதிவு செய்யப்பட்டுள்ளன, அவர்களில் 5 பேர் மருத்துவமனைக்குக் கொண்டு வரப்பட்டபோது இறந்து இருந்தனர். இது தேசிய இறப்பு எண்ணிக்கையை 1,111-ஆக கொண்டுவருகிறது.\nஇன்றைய இறப்புகள், சிலாங்கூர் (4), கோலாலம்பூர் (3), சரவாக் (2), சபா (1), ஜொகூர் (1) என பதிவாகியுள்ளன. இறந்தவர்களில் மூன்று பேர் மலேசியர் \xa0அல்லாதவர்கள்.\nஇன்று 3,085 நோயாளிகள் குணமடைந்து வீடு திரும்பியுள்ளனர். அவசரப் பிரிவில் 201 பேர் சிகிச்சை பெற்று வருகின்றனர், அவர்களில் 93 பேருக்குச் சுவாசக் கருவியின் உத

In [34]:
with open('malaysiaindru.my.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f) as fopen:
            data = json.load(fopen)
            fopen_l.write(f'{json.dumps(data)}\n')